In [15]:
%load_ext autoreload
%autoreload 2

from Shared.shared import *
from Shared.specific_CNB_sim import *

import pandas as pd

sim_name = f"Dopri5_1k"
sim_folder = f"sim_output/{sim_name}"
fig_folder = f"figures_local/{sim_name}"
Cl_folder = f"Shared/Cls"
nu_m_range = jnp.load(f"{sim_folder}/neutrino_massrange_eV.npy")
nu_m_picks = jnp.array([0.01, 0.05, 0.1, 0.2, 0.3])*Params.eV
simdata = SimData(sim_folder)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Earth-Sun distances

In [ ]:
# Load the Excel file into a pandas DataFrame
df = pd.read_excel('Data/Earth-Sun_distances.xlsx')


ES_distances = jnp.array(df.iloc[:, 1::2].apply(pd.to_numeric, errors='coerce')\
                                  .stack().reset_index(drop=True).tolist())[:-1]
ES_dists_kpc = ES_distances*Params.AU/Params.kpc

# print(ES_distances[jnp.array([0,-1])])
#? first and last day are the same day? since there are 366 elements?

days = jnp.arange(1,len(ES_distances)+1)

print(ES_distances.shape, days.shape)

print(ES_distances[0], ES_dists_kpc[0])

## Integration steps

In [16]:
from astropy.time import Time
from scipy.integrate import quad

def z_at_age(age):
    # Function to calculate redshift for a given age of the universe
    def age_diff(z):
        return cosmo.age(z).to(apu.year).value - age
    
    # Use root_scalar to find the redshift that gives the specified age
    sol = root_scalar(age_diff, bracket=[0, 10], method='brentq', xtol=1e-12, rtol=1e-12)
    return sol.root

def get_redshift_array():
    # Today's date
    today = Time.now()
    
    days = 365

    # Initialize an array to store redshift values with 64-bit precision
    redshifts = jnp.zeros(days, dtype=jnp.float64)
    
    # Age of the universe today
    age_today = cosmo.age(0).to(apu.year).value
    
    # Calculate redshift for each day in the past year
    for day in range(days):
        # Calculate the date for each day of the past year
        past_date = today - day * apu.day
        # Calculate the lookback time in years
        lookback_time = (today - past_date).to(apu.year).value
        # Calculate the age of the universe at this lookback time
        age_of_universe_then = age_today - lookback_time
        # Find the redshift corresponding to this age of the universe
        redshift = z_at_age(age_of_universe_then)
        # Store the redshift value in the array
        redshifts = redshifts.at[day].set(redshift)
    
    return redshifts


def s_of_z(z, args):
    """
    Convert redshift to time variable s with eqn. 4.1 in Mertsch et al.
    (2020), keeping only Omega_M and Omega_L in the Hubble eqn. for H(z).

    Args:
        z (float): redshift

    Returns:
        float: time variable s (in [seconds] if 1/H0 factor is included)
    """    

    def s_integrand(z):        

        # We need value of H0 in units of 1/s.
        H0_val = args.H0/(1/args.s)
        a_dot = np.sqrt(args.Omega_M*(1.+z)**3 + args.Omega_L)/(1.+z)*H0_val
        s_int = 1./a_dot

        return s_int

    s_of_z, _ = quad(s_integrand, 0., z)

    return jnp.float64(s_of_z)


# Get the redshift array for the past year
z_int_steps_1year = get_redshift_array()
print(z_int_steps_1year.shape)

s_int_steps_1year = jnp.array([s_of_z(z, Params()) for z in z_int_steps_1year])
print(s_int_steps_1year.shape)

jnp.save(f"{sim_folder}/z_int_steps_1year.npy", z_int_steps_1year)
jnp.save(f"{sim_folder}/s_int_steps_1year.npy", s_int_steps_1year)

(365,)
(365,)
